# Stage 2: Preparing Visualization Dataset

This notebook contains the instructions to prepare a dataset for a bubble map data visualization (made in Power BI) which uses color encoding to identify which high schools teach Computer Science and which don't. 

This dataset will consist of three key types of columns: 

1. **School Course Statistics** such as whether it teaches comptuer science or not, how many computer science courses does it teach, how many computer science enrollments were recorded for the year. 

2. **School Information** such as the School Name, Email Address, Principal Name, Phone Number

3. **School GeoData** such as the latitude and longitude

The datasets imported in this notebook will be from the `data/labelled_data` path. These datasets were prepared by us in the previous Stage 1: Labelling Courses notebook. 

These instructions will be of help when you want to prepare data for a visualization in future years. The original file format provided by OSPI may slightly change but this is a good reference point to see our method. You may need to tweak the names of the files being imported and the column names if they have changed with time.

*Note: For new Panda users, a dataframe is a dataset table which consists of columns and rows. We will be using this term frequently when discussing our process.*

## Part 1: Setup

In this step we will import each of the necessary packages for our data preparation and data wrangling.  

We will be using the following packages: 

1. **CSV:** to read in csv files
2. **Pandas:** for data wrangling - this involves reshaping, merging, concatting(adding 2 dataframes), adding and removing columns, renaming columns, grouping and summarizing. 
3. **Numpy:** for mathematical operations - this involves setting the data types of columns and setting defaults values of columns
4. **Altair:** for data visualizations. We will be using this package to quickly create a rapid data visualization to test our data visualization dataset. 

In [1]:
#Setup (Importing Packages) 
import csv #to read in csv files
import pandas as pd #for data wrangling
import numpy as np  #for mathematical operations
import altair as alt #for data visualization
import warnings

## Part 2: Preparing the CIP Course Dataset

In this part we will be preparing the CIP Course Dataset to showcase the total students in each CIP course. 

**1. In this step we are importing `cip_course_statistics_2017.csv` file using the pandas read_csv method and saving it as a dataframe.** 

This dataframe contains a list of the CIP courses taught at middle schools and high schools in the state of Washington. Please note that this list is partially complete as data was not available for all Washington schools in the year 2017.

*Note: That you will need to edit the file name to match the year of the file you are aiming to process a file for. So for example you would change the file name from 'cip_course_statistics_2017.csv' to 'cip_course_statistics_2018.csv'*

In [2]:
#Importing and Saving Student Results for CIP Courses
#Note: this is where you will want to change the file name for the new CIP Student Results Dataset
cip_courses  = pd.read_csv("data/labelled_data/CIP_Data/cip_course_statistics_2017.csv")

**2. The head of the dataframe (the first 5 rows) has been printed for you to get a better understanding of the data.**

*Note: If you want to see all the data just type `cip_courses` in a new cell block and click on Shift and Enter at the same time.*

**Here is a quick explanation on what each column is.**

| **COLUMN NAME** | **COLUMN DESCRIPTION** |
| ----------- | ----------- |
| **DistrictCode:** | Code of the School District in which the school is (e.g. 2420) |
| **DistrictName:** | Name of the School District in which the school is (e.g. Asotin-Anatone School District) |
| **SchoolCode:** |Code of the School (e.g. 2434) |
| **SchoolName:**v| Name of the School (e.g. Asotin Jr Sr High) |
| **term:** | Which semester was the course in (e.g. SEM1) |
| **cipcode:** | National Course Code under which this class falls |
|  **courseTitle:** | Title of the course (e.g. AP Computer Science Principles) |
| **letterGrade:** | LetterGrade (e.g. A, A-, B, etc.) |
| **count:** | total students who received that letterGrade in the course |
| **cs_course:** | whether the course is a computer science course or not |

In [3]:
# Printing the head of the dataframe
cip_courses.head(5)

,DistrictCode,DistrictName,SchoolCode,SchoolName,term,cipcode,courseTitle,letterGrade,count,cs_course
0,2420,Asotin-Anatone School District,2434,Asotin Jr Sr High,SEM1,110201,AP Computer Science Principles,A,3,yes
1,2420,Asotin-Anatone School District,2434,Asotin Jr Sr High,SEM1,110201,CSS ENGINEERING,A-,1,yes
2,2420,Asotin-Anatone School District,2434,Asotin Jr Sr High,SEM1,110201,CSS ENGINEERING,A,3,yes
3,2420,Asotin-Anatone School District,2434,Asotin Jr Sr High,SEM1,110201,CSS ENGINEERING,D,1,yes
4,2420,Asotin-Anatone School District,2434,Asotin Jr Sr High,SEM2,110201,AP Computer Science Principles,A,3,yes


**3. In the `cip_courses` dataframe each letterGrade statistic of a course is it's own row. This increases the total number of rows as well as makes it difficult for us to see the statistics for a course easily in one row. Therefore we are going to spread the data such that each letter grade becomes its own column. This will also help us in calculating the total students easily. I have printed the head of the `cip_courses` dataframe for you to easily see the changes made.**

In [4]:
#Reshaping (Spreading) - lettergrades are becoming columns
cip_courses = pd.pivot_table(cip_courses, index = ['DistrictCode','DistrictName','SchoolCode','SchoolName','term','cipcode','courseTitle', 'cs_course'], columns = 'letterGrade', values = 'count')

# We do the NA replacement to 0 step below because each course 
# did not always have a count of 1 or more for each letterGrade. 
# Because of this some letterGrade columns for a course filled the cell value as NA instead of 0. 

#Fill NA for Letter Grades to 0
cip_courses = cip_courses.fillna(0)

#Strip the extra space at the start and end of column names
cip_courses.columns = cip_courses.columns.str.rstrip()

#List of Column Names
cols = ['A', 'A-', 'B', 'B+', 'B-', 'C','C+', 'C-', 'CR', 'D', 'D+', 'E', 'F', 'N', 'NC', 'P', 'S', 'U', 'W']

#Convert the Columns listed in the `cols` list to Integer DataType
cip_courses[cols] = cip_courses[cols].applymap(np.int64)

cip_courses.head()

letterGrade                                                                                                        A  \
DistrictCode DistrictName            SchoolCode SchoolName                 term cipcode courseTitle    cs_course       
1147         Othello School District 3015       Othello High School        SEM2 110801  DIGITAL DESIGN no         27   
1158         Lind School District    2903       Lind-Ritzville High School SEM1 110103  TECHNOLOGY 1A  no          4   
                                                                                110801  PHOTOGRAPHY    no          2   
                                                                           SEM2 110103  TECHNOLOGY 1B  no          3   
                                                                                110801  PHOTOGRAPHY    no          1   

letterGrade                                                                                                       A-  \
DistrictCode DistrictName            SchoolCode SchoolName                 term cipcode courseTitle    cs_course       
1147         Othello School District 3015       Othello High School        SEM2 110801  DIGITAL DESIGN no          5   
1158         Lind School District    2903       Lind-Ritzville High School SEM1 110103  TECHNOLOGY 1A  no          2   
                                                                                110801  PHOTOGRAPHY    no          1   
                                                                           SEM2 110103  TECHNOLOGY 1B  no          0   
                                                                                110801  PHOTOGRAPHY    no          1   

letterGrade                                                                                                       B  \
DistrictCode DistrictName            SchoolCode SchoolName                 term cipcode courseTitle    cs_course      
1147         Othello School District 3015       Othello High School        SEM2 110801  DIGITAL DESIGN no         3   
1158         Lind School District    2903       Lind-Ritzville High School SEM1 110103  TECHNOLOGY 1A  no         1   
                                                                                110801  PHOTOGRAPHY    no         0   
                                                                           SEM2 110103  TECHNOLOGY 1B  no         0   
                                                                                110801  PHOTOGRAPHY    no         0   

letterGrade                                                                                                       B+  \
DistrictCode DistrictName            SchoolCode SchoolName                 term cipcode courseTitle    cs_course       
1147         Othello School District 3015       Othello High School        SEM2 110801  DIGITAL DESIGN no          3   
1158         Lind School District    2903       Lind-Ritzville High School SEM1 110103  TECHNOLOGY 1A  no          0   
                                                                                110801  PHOTOGRAPHY    no          0   
                                                                           SEM2 110103  TECHNOLOGY 1B  no          0   
                                                                                110801  PHOTOGRAPHY    no          1   

letterGrade                                                                                                       B-  \
DistrictCode DistrictName            SchoolCode SchoolName                 term cipcode courseTitle    cs_course       
1147         Othello School District 3015       Othello High School        SEM2 110801  DIGITAL DESIGN no          3   
1158         Lind School District    2903       Lind-Ritzville High School SEM1 110103  TECHNOLOGY 1A  no          0   
                                                                                110801  PHOTOGRAPHY    no          0   
                                           

**4. We will now add a column called total students in the course which sums up the count for each lettergrade. I have printed the head of the column for you to see the new `total_students` column which has been added.**

In [5]:
#Adding Column Stating Total Students in Course
cip_courses['total_students'] = cip_courses['A'] + cip_courses['A-'] + cip_courses['B'] + cip_courses['B+']+ cip_courses['B-']+ cip_courses['C']+ cip_courses['C-']+ cip_courses['C+']+ cip_courses['CR']+ cip_courses['D']+ cip_courses['D+']+ cip_courses['E']+ cip_courses['F']+ cip_courses['N']+ cip_courses['NC']+ cip_courses['P']+ cip_courses['S']+ cip_courses['U']+ cip_courses['W']                

#Resetting Index of Dataframe.
cip_courses = cip_courses.reset_index()

#Showing the head of the dataframe
cip_courses.head()

letterGrade,DistrictCode,DistrictName,SchoolCode,SchoolName,term,cipcode,courseTitle,cs_course,A,A-,...,D+,E,F,N,NC,P,S,U,W,total_students
0,1147,Othello School District,3015,Othello High School,SEM2,110801,DIGITAL DESIGN,no,27,5,...,0,0,3,0,0,0,0,0,0,60
1,1158,Lind School District,2903,Lind-Ritzville High School,SEM1,110103,TECHNOLOGY 1A,no,4,2,...,0,0,0,0,0,0,0,0,0,12
2,1158,Lind School District,2903,Lind-Ritzville High School,SEM1,110801,PHOTOGRAPHY,no,2,1,...,0,0,2,0,0,0,0,0,0,7
3,1158,Lind School District,2903,Lind-Ritzville High School,SEM2,110103,TECHNOLOGY 1B,no,3,0,...,0,0,3,0,0,0,0,0,0,10
4,1158,Lind School District,2903,Lind-Ritzville High School,SEM2,110801,PHOTOGRAPHY,no,1,1,...,0,0,0,0,0,0,0,0,0,5


## Part 3: Preparing the State Course Code DataSet
In this step, we are preparing the State Course Code Dataset to showcase the total students in each state course code course.

**1. In this step we are importing the `state_course_code_statistics_2017.csv` using the read_csv method of the Pandas package and saving it as a dataframe.**

This dataframe is a list of state course code courses taught at schools in the state of Washington. Please note that this list is partially complete as data is not available for all Washington state schools as of 2017. 

*Note: That you will need to edit the file name to match the year of the file you are aiming to process a file for. So for example you would change the file name from 'state_course_code_statistics_2017.csv' to 'state_course_code_statistics_2018.csv'*

In [6]:
#Importing and Saving Student Results for State Courses
#Note: this is where you will want to change the file name for the new SCC Student Results Dataset
scc_courses = pd.read_csv("data/labelled_data/State_Course_Code_Data/state_course_code_statistics_2017.csv")

**2. The head of the dataframe (the first 5 rows) has been printed for you to get a better understanding of the data.**

*Note: If you want to see all the data just type `scc_courses` in a new cell block and click on Shift and Enter at the same time.*

**Here is a quick explanation on what each column is.**

| **COLUMN NAME** | **COLUMN DESCRIPTION** |
| ----------- | ----------- |
| **DistrictCode:** | Code of the School District in which the school is (e.g. 2420) |
| **DistrictName:** | Name of the School District in which the school is (e.g. Asotin-Anatone School District) |
| **SchoolCode:** |Code of the School (e.g. 2434) |
| **SchoolName:**v| Name of the School (e.g. Asotin Jr Sr High) |
| **term:** | Which semester was the course in (e.g. SEM1) |
| **stateCourseCodeId:** | State Course Code under which this class falls |
|  **courseTitle:** | Title of the course (e.g. AP Computer Science Principles) |
| **letterGrade:** | LetterGrade (e.g. A, A-, B, etc.) |
| **count:** | total students who received that letterGrade in the course |
| **cs_course:** | whether the course is a computer science course or not |

In [7]:
#printing the head of scc_courses
scc_courses.head()

,DistrictCode,DistrictName,SchoolCode,SchoolName,term,stateCourseCodeId,courseTitle,letterGrade,count,cs_course
0,17407,Riverview School District,3524,Cedarcrest High School,SEM1,837,PRG/GAMES/SIM A,A-,2,no
1,17407,Riverview School District,3524,Cedarcrest High School,SEM1,837,PRG/GAMES/SIM A,A,6,no
2,17407,Riverview School District,3524,Cedarcrest High School,SEM1,837,PRG/GAMES/SIM A,B,1,no
3,17407,Riverview School District,3524,Cedarcrest High School,SEM1,837,PRG/GAMES/SIM A,B+,1,no
4,17407,Riverview School District,3524,Cedarcrest High School,SEM1,837,PRG/GAMES/SIM A,C,1,no


**3. In the `scc_courses` dataframe each letterGrade statistic of a course is it's own row. This increases the total number of rows as well as makes it difficult for us to see the statistics for a course easily in one row. Therefore we are going to spread the data such that each letter grade becomes its own column. This will also help us in calculating the total students easily. I have printed the head of the `scc_courses` dataframe for you to easily see the changes made.**

In [8]:
#Reshaping (Spreading) - lettergrades are becoming columns
scc_courses = pd.pivot_table(scc_courses, index = ['DistrictCode','DistrictName','SchoolCode','SchoolName','term','stateCourseCodeId','courseTitle', 'cs_course'], columns = 'letterGrade', values = 'count')

# We do the NA replacement to 0 step below because each course 
# did not always have a count of 1 or more for each letterGrade. 
# Because of this some letterGrade columns for a course filled the cell value as NA instead of 0. 

#Fill NA for Letter Grades to 0 
scc_courses = scc_courses.fillna(0)

#Strip the extra space at the start and end of column names
scc_courses.columns = scc_courses.columns.str.rstrip()

#List of Column Names
cols = ['A', 'A-', 'B', 'B+', 'B-', 'C','C+', 'C-', 'CR', 'D', 'D+', 'E', 'F', 'N', 'NC', 'P', 'S', 'U', 'W']

#Convert Columns in List to Integer DataType
scc_courses[cols] = scc_courses[cols].applymap(np.int64)

#Printing the Head
scc_courses.head()

letterGrade                                                                                                                         A  \
DistrictCode DistrictName              SchoolCode SchoolName                       term stateCourseCodeId courseTitle   cs_course       
1109         Washtucna School District 3075       Washtucna Elementary/High School SEM1 2309              10 ENGLISH    no          1   
                                                                                   SEM2 2309              10 ENGLISH    no          3   
1147         Othello School District   3015       Othello High School              SEM1 2696              DIGITOOLS     no          4   
                                                                                                          DIGITOOLS C/D no         11   
                                                                                   SEM2 2696              DIGITOOLS     no         11   

letterGrade                                                                                                                        A-  \
DistrictCode DistrictName              SchoolCode SchoolName                       term stateCourseCodeId courseTitle   cs_course       
1109         Washtucna School District 3075       Washtucna Elementary/High School SEM1 2309              10 ENGLISH    no          0   
                                                                                   SEM2 2309              10 ENGLISH    no          0   
1147         Othello School District   3015       Othello High School              SEM1 2696              DIGITOOLS     no          1   
                                                                                                          DIGITOOLS C/D no          1   
                                                                                   SEM2 2696              DIGITOOLS     no          4   

letterGrade                                                                                                                        B  \
DistrictCode DistrictName              SchoolCode SchoolName                       term stateCourseCodeId courseTitle   cs_course      
1109         Washtucna School District 3075       Washtucna Elementary/High School SEM1 2309              10 ENGLISH    no         0   
                                                                                   SEM2 2309              10 ENGLISH    no         0   
1147         Othello School District   3015       Othello High School              SEM1 2696              DIGITOOLS     no         8   
                                                                                                          DIGITOOLS C/D no         5   
                                                                                   SEM2 2696              DIGITOOLS     no         3   

letterGrade                                                                                                                        B+  \
DistrictCode DistrictName              SchoolCode SchoolName                       term stateCourseCodeId courseTitle   cs_course       
1109         Washtucna School District 3075       Washtucna Elementary/High School SEM1 2309              10 ENGLISH    no          0   
                                                                                   SEM2 2309              10 ENGLISH    no          1   
1147         Othello School District   3015       Othello High School              SEM1 2696              DIGITOOLS     no          0   
                                                                                                          DIGITOOLS C/D no          1   
                                                                                   SEM2 2696              DIGITOOLS     no          5   

letterGrade                                                                                                                        B-  \
DistrictCode DistrictName     

**4. We will now add a column called total students in the course which sums up the count for each lettergrade. I have printed the head of the column for you to see the new `total_students` column which has been added.**

In [9]:
#Adding Column Stating Total Students in Course
scc_courses['total_students'] = scc_courses['A'] + scc_courses['A-'] + scc_courses['B'] + scc_courses['B+']+ scc_courses['B-']+ scc_courses['C']+ scc_courses['C-']+ scc_courses['C+']+ scc_courses['CR']+ scc_courses['D']+ scc_courses['D+']+ scc_courses['E']+ scc_courses['F']+ scc_courses['N']+ scc_courses['NC']+ scc_courses['P']+ scc_courses['S']+ scc_courses['U']+ scc_courses['W']                

#Resetting Index of Dataframe
scc_courses = scc_courses.reset_index()

#Showing the head of the dataframe
scc_courses.head()

letterGrade,DistrictCode,DistrictName,SchoolCode,SchoolName,term,stateCourseCodeId,courseTitle,cs_course,A,A-,...,D+,E,F,N,NC,P,S,U,W,total_students
0,1109,Washtucna School District,3075,Washtucna Elementary/High School,SEM1,2309,10 ENGLISH,no,1,0,...,0,0,0,0,0,0,0,0,0,3
1,1109,Washtucna School District,3075,Washtucna Elementary/High School,SEM2,2309,10 ENGLISH,no,3,0,...,0,0,0,0,0,0,0,0,0,5
2,1147,Othello School District,3015,Othello High School,SEM1,2696,DIGITOOLS,no,4,1,...,2,0,3,0,0,0,0,0,0,28
3,1147,Othello School District,3015,Othello High School,SEM1,2696,DIGITOOLS C/D,no,11,1,...,1,0,1,0,0,0,0,0,2,25
4,1147,Othello School District,3015,Othello High School,SEM2,2696,DIGITOOLS,no,11,4,...,2,0,4,0,0,0,0,0,2,53


## About Testing
This part onwards, there will be a test written after key parts to test whether any rows have been unintentionally dropped during the merging and concatting actions. There will also be tests written to check if any important columns have any missing data. Please pay attention to the test results printed after each cell. The head of a dataframe will only be printed if the test is passed. These may be simple if else tests but it is very important to ensure all tests get passed. 

## Part 4: Combining CIP Course Data and State Course Data

In this step, we will be combining the CIP Code Course dataframe (`cip_courses`) and State Course Code Course dataframe (`scc_courses`) into one dataframe. 

In [10]:
#Combining the State Course and CIP Course Data using the concat method of Pandas
all_courses = pd.concat([cip_courses, scc_courses], sort=False)

####################################################################################################
### ALL COURSES TEST:
### This test checks if the total rows of all courses is equal 
### to the sum of rows of the CIP courses and SCC courses dataframes

# Saving Test variable for test at the end
all_courses_test = (len(all_courses)== (len(cip_courses) + len(scc_courses)))

if(all_courses_test):
    print("PASSED TEST: ALL COURSES TEST")

else:
    print("NOT PASSED: ALL COURSES TEST")    

# Printing the head of the dataframe
all_courses.head()

PASSED TEST: ALL COURSES TEST


,DistrictCode,DistrictName,SchoolCode,SchoolName,term,cipcode,courseTitle,cs_course,A,A-,...,E,F,N,NC,P,S,U,W,total_students,stateCourseCodeId
0,1147,Othello School District,3015,Othello High School,SEM2,110801.0,DIGITAL DESIGN,no,27,5,...,0,3,0,0,0,0,0,0,60,NaN
1,1158,Lind School District,2903,Lind-Ritzville High School,SEM1,110103.0,TECHNOLOGY 1A,no,4,2,...,0,0,0,0,0,0,0,0,12,NaN
2,1158,Lind School District,2903,Lind-Ritzville High School,SEM1,110801.0,PHOTOGRAPHY,no,2,1,...,0,2,0,0,0,0,0,0,7,NaN
3,1158,Lind School District,2903,Lind-Ritzville High School,SEM2,110103.0,TECHNOLOGY 1B,no,3,0,...,0,3,0,0,0,0,0,0,10,NaN
4,1158,Lind School District,2903,Lind-Ritzville High School,SEM2,110801.0,PHOTOGRAPHY,no,1,1,...,0,0,0,0,0,0,0,0,5,NaN


## Part 5: Making a lite version of all_courses dataframe

In this cell below, we are making a lite version of the all courses dataframe by selecting only the columns of concern to us which are: ``"DistrictCode","SchoolCode", "SchoolName", "courseTitle", "cs_course", "total_students"``. The head of the dataframe has been printed for you to see the lite version of this dataframe.

In [11]:
# Making a small version of the all courses file with only the necessary columns
all_courses_lite = all_courses.copy()

#Selecting only columns of concern
all_courses_lite = all_courses_lite[["DistrictCode","SchoolCode", "SchoolName", "courseTitle", "cs_course", "total_students"]]


# Test 
all_courses_lite_test = (len(all_courses) == len(all_courses_lite))

if(all_courses_lite_test):
    print("PASSED TEST: ALL COURSES LITE")
else: 
    print("NOT PASSED TEST: ALL COURSES LITE")

all_courses_lite.head()

PASSED TEST: ALL COURSES LITE


,DistrictCode,SchoolCode,SchoolName,courseTitle,cs_course,total_students
0,1147,3015,Othello High School,DIGITAL DESIGN,no,60
1,1158,2903,Lind-Ritzville High School,TECHNOLOGY 1A,no,12
2,1158,2903,Lind-Ritzville High School,PHOTOGRAPHY,no,7
3,1158,2903,Lind-Ritzville High School,TECHNOLOGY 1B,no,10
4,1158,2903,Lind-Ritzville High School,PHOTOGRAPHY,no,5


## Part 6: Listing Schools on which we have course data

**As discussed earlier we do not have course information on all schools in the state of Washington. In the cell below we are making a dataframe which contains the names of all the schools we have course information on. We have printed the head of the dataframe for your easy understanding. Below we have printed the head of the dataframe for easy viewing.**

In [13]:
#Copy of all_courses_lite is made for the list of known schools
known_schools = all_courses_lite.copy()

# We only select the most important columns needed
known_schools = known_schools[["SchoolName", "DistrictCode", "SchoolCode"]]

# Since our dataframe has multiple copies of schools we will drop the duplicates, such that each row has a unique School Code
# WARNING: We use the school code when dropping duplicates instead of dropping by School Name as some schools have the same name
known_schools = known_schools.drop_duplicates(['SchoolCode'])

# We reset the index for easy reading and to ensure our rows are in sequential order. 
# Note: when we reindex, a new column called index is added with the old indexes, 
# Note(contd) we drop this column as it is not necessary
known_schools = known_schools.reset_index().drop(columns = ['index'])

# Printing the head of known schools
known_schools.head()

,SchoolName,DistrictCode,SchoolCode
0,Othello High School,1147,3015
1,Lind-Ritzville High School,1158,2903
2,Ritzville High School,1160,2132
3,Asotin Jr Sr High,2420,2434
4,Mid-Columbia Parent Partnership,3017,1941


## Part 7: Listing Schools which teach computer science
**In the cell below, we are creating a dataframe which consists of a list of schools which teach computer science, the total computer science classes they teach, and the total number of students they have enrolled in computer science. We do this by filtering out for rows which have the the cs_course column cell value as "yes" (in the all_courses dataframe).**

In [14]:
# Filtering to keep only Computer Science courses
cs_results = all_courses.loc[all_courses["cs_course"] == "yes"]

# Grouping by high school and summarizing for the count of computer science classes taught
cs_schools = cs_results.groupby(['SchoolCode','SchoolName']).agg({'cs_course': 'count', 'total_students': 'sum'})

# Adding Column to say School Teaches Computer Science
cs_schools["school_teaches_cs"] = "Teaches Computer Science"

# Resetting the Index after grouping by
cs_schools = cs_schools.reset_index()

# Renaming column to state total computer science courses taught in that year
cs_schools = cs_schools.rename(columns = {'cs_course': 'total_cs_courses', 'total_students': 'yearly_enrolled_in_cs'})

# Printing head of schools which teach computer science dataframe
cs_schools.head()

,SchoolCode,SchoolName,total_cs_courses,yearly_enrolled_in_cs,school_teaches_cs
0,1519,Edmonds eLearning Academy,1,1,Teaches Computer Science
1,1547,Middle College High School,4,58,Teaches Computer Science
2,1627,Yelm Extension School,2,3,Teaches Computer Science
3,1628,Dishman Hills High School,6,158,Teaches Computer Science
4,1640,Puyallup Online Academy/POA,2,6,Teaches Computer Science


## Part 8: Listing Schoools which don't teach computer science
**In the cell below, we are creating a dataframe which consists of a list of schools which do not teach computer science, the total computer science classes they teach (in this case 0), and the total number of students they have enrolled in computer science (in this case 0). We do this by retaining a list of schools which are only present in the `known_schools dataframe` and not in the `cs_schools` dataframe. Below we have printed the head of this dataframe for your easy viewing.**

In [15]:
non_cs_schools = known_schools[~known_schools.SchoolCode.isin(cs_schools.SchoolCode)]

#Adding column to say it teaches 0 cs courses
non_cs_schools["total_cs_courses"] = 0

#Adding column to say it has 0 students enrolled in CS
non_cs_schools["yearly_enrolled_in_cs"] = 0

#Adding column to say that School does not teache CS
non_cs_schools["school_teaches_cs"] = "Doesn't Teach Computer Science"

#Resetting Index
non_cs_schools = non_cs_schools.reset_index().drop(columns = ['index', 'DistrictCode'])

#Selecting columns to keep
#non_cs_schools = non_cs_schools[['SchoolName','SchoolCode', 'total_cs_courses', 'yearly_enrolled_in_cs', "school_teaches_cs"]]

#Printing schools which do not teach CS
non_cs_schools.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

,SchoolName,SchoolCode,total_cs_courses,yearly_enrolled_in_cs,school_teaches_cs
0,Othello High School,3015,0,0,Doesn't Teach Computer Science
1,Lind-Ritzville High School,2903,0,0,Doesn't Teach Computer Science
2,Prosser High School,2508,0,0,Doesn't Teach Computer Science
3,Richland High School,3511,0,0,Doesn't Teach Computer Science
4,Entiat Middle and High School,3317,0,0,Doesn't Teach Computer Science


## Part 9: Listing Statistics of Known Schools

In this step, we are going to list the statistics we have about the state of computer education in schools we have information on. We do this by concatting (adding) the newly created `cs_schools` and `non_cs_schools` dataframes made in Parts 6 and 7.

In [17]:
#Adding CS and Non CS Schools Data Frames
known_schools_stats = pd.concat([cs_schools, non_cs_schools]) 

#Resetting Index
known_schools_stats = known_schools_stats.reset_index().drop(columns = ['index'])

#
known_schools_stats_test = (len(known_schools_stats) == (len(cs_schools) + len(non_cs_schools)))
if(known_schools_stats_test):
    print("PASSED TEST: KNOWN SCHOOLS STATS TEST")
else:
    print("NOT PASSED TEST: KNOWN SCHOOLS STATS TEST")

#Printing the head of the dataframe
known_schools_stats.head()

PASSED TEST: KNOWN SCHOOLS STATS TEST


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


,SchoolCode,SchoolName,school_teaches_cs,total_cs_courses,yearly_enrolled_in_cs
0,1519,Edmonds eLearning Academy,Teaches Computer Science,1,1
1,1547,Middle College High School,Teaches Computer Science,4,58
2,1627,Yelm Extension School,Teaches Computer Science,2,3
3,1628,Dishman Hills High School,Teaches Computer Science,6,158
4,1640,Puyallup Online Academy/POA,Teaches Computer Science,2,6


## Part 10: List of All High Schools in Washington State

In this step we are importing the `High_Schools_WA_Information.csv` file using the Panda package's read_csv method and saving it as a dataframe called `high_schools`. This file was sourced from the [OSPI School Directory website](https://eds.ospi.k12.wa.us/directoryeds.aspx). 

**Note:** This file modified in the following ways: 

    - Only the most relevant columns were kept for our visualization dataset were kept. Those were: 'LEACode', 'LEAName', 'SchoolCode', 'SchoolName', 'LowestGrade','HighestGrade', 'PrincipalName', 'Email','Phone', 'OrgCategoryList','GradeCategory', and 'City'. 

    - We only kept the schools which have their highest grade as 9, 10, 11 or 12. Some of these schools are alternate schools, jails, detention centres and learning centres. Do not be surprised by this. We have retained these schools so as to be inclusive. 

    - It was missing the names of the last three schools listed in this csv file. I added these manually by doing a google search for information. Please beware that the quality of the data from OSPI is not 100% complete always. 

Below I have listed out the description of each column name for easy understanding.

| **Column Name** | **Column Description** | 
| ----------- | ----------- |
| LEACode | Local Education Agency	Code e.g. 3346|
| LEAName | Local Education Agency	Name e.g. Colfax School District|
| SchoolCode | School Code of the School in Washington State e.g. 3366|
| SchoolName | Name of the School e.g. Colfax High School|
| LowestGrade | Lowest Grade in the School e.g. 7 |
| HighestGrade | Highest Grade in the School e.g. 12|
| Principal Name | Name of the School Principal e.g. David Gibb |
| Email | Email of the Principal e.g. david.gibb@csd300.com	|
| Phone |Phone Number of the Principal e.g. 509.830.2347	|
|OrgCategoryList |Type of Category the School falls under e.g. Public School, Regular School	| 
|Grade Category | Type of School e.g. High School, K-12, etc.) |
| City | City Name e.g. Colfax |

In [18]:
# importing and saving the High_Schools_WA_Information CSV file
high_schools = pd.read_csv("data/labelled_data/School_Data/High_Schools_WA_Information.csv")
high_schools.head()

,LEACode,LEAName,SchoolCode,SchoolName,LowestGrade,HighestGrade,PrincipalName,Email,Phone,OrgCategoryList,GradeCategory,City
0,38300,Colfax School District,3366,Colfax High School,7,12,David Gibb,david.gibb@csd300.com,509.830.2347,"Public School, Regular School",High School,Colfax
1,38301,Palouse School District,2634,Palouse High School,9,12,Mike Jones,mjones@garpal.net,509.878.1921,"Public School, Regular School",High School,Palouse
2,38306,Colton School District,2588,Colton School,PK,12,Tim Casey,tcasey@colton.k12.wa.us,509.229.3386,"Public School, Regular School",PK-12,Colton
3,38320,Rosalia School District,3204,Rosalia Elementary & Secondary School,PK,12,Matthew McLain,mmclain@rosaliaschools.org,509.523.3061,"Public School, Regular School",PK-12,Rosalia
4,38322,St. John School District,3068,St John/Endicott High,9,12,Mark Purvine,mpurvine@stjohn.wednet.edu,509.648.3336,"Public School, Regular School",High School,Saint John


## Part 11: Listing High Schools we have Statistics on

The list of statistics we have is for all types of schools in the state of Washington. We only want statistics for schools which have a grade of 9 and above. Therefore we are going to make a new dataframe called `known_high_school_stats` which will retain only the statistics of high schools in `known_schools_stats`.

In [19]:
# Making a copy of the known_school_stats dataframe
known_high_school_stats = known_schools_stats.copy()

# We are only retaining school rows which are high schools
known_high_school_stats = known_high_school_stats[known_high_school_stats.SchoolCode.isin(high_schools.SchoolCode)]

# Resetting Index
known_high_school_stats = known_high_school_stats.reset_index().drop(columns = ['index'])

# Printing the head of the new dataframe
known_high_school_stats.head()

,SchoolCode,SchoolName,school_teaches_cs,total_cs_courses,yearly_enrolled_in_cs
0,1519,Edmonds eLearning Academy,Teaches Computer Science,1,1
1,1547,Middle College High School,Teaches Computer Science,4,58
2,1627,Yelm Extension School,Teaches Computer Science,2,3
3,1628,Dishman Hills High School,Teaches Computer Science,6,158
4,1640,Puyallup Online Academy/POA,Teaches Computer Science,2,6


## Part 12: Listing High Schools we do not have statistics on

As mentioned earlier, we do not have course information on all schools. It is important for us to also include the schools we do not have information on. This is so that it is easy to identify which Local Education Agency's are not collecting and providing information on their schools.


In [20]:
# Making a copy of the high_schools dataframe
unknown_high_schools = high_schools.copy()

# We are only retaining the high schools on which we have no information
unknown_high_schools = unknown_high_schools[~unknown_high_schools.SchoolCode.isin(known_high_school_stats.SchoolCode)]

# We are only keeping the SchoolCode and SchoolName columns as only those are needed for the overall Washington State High School statistics 
# dataframe we are making at the moment. 
unknown_high_schools = unknown_high_schools[["SchoolCode", "SchoolName"]]

# We are adding the total_cs_courses column and setting the value as NA 
# as we have no information on whether the school teaches computer science or not
unknown_high_schools["total_cs_courses"] = np.nan

# We are adding the yearly_enrolled_in_cs column and setting the value as NA 
# as we have no information on whether the school teaches computer science or not

unknown_high_schools["yearly_enrolled_in_cs"] = np.nan

# We are adding the school_teaches_cs column and setting the value as NA 
# as we have no information on whether the school teaches computer science or not

unknown_high_schools["school_teaches_cs"] = "No Information Available"

# Resetting Index
unknown_high_schools = unknown_high_schools.reset_index().drop(columns = ['index'])

# Printing the head
unknown_high_schools.head()

,SchoolCode,SchoolName,total_cs_courses,yearly_enrolled_in_cs,school_teaches_cs
0,3204,Rosalia Elementary & Secondary School,NaN,NaN,No Information Available
1,4040,West Valley Jr High,NaN,NaN,No Information Available
2,1910,Marysville SD Special,NaN,NaN,No Information Available
3,1904,Parent Partnership,NaN,NaN,No Information Available
4,1932,Columbia Virtual Academy,NaN,NaN,No Information Available


## Part 13: Combining High Schools we have statistics on with High Schools we do not have statistics on

**We are now combining the dataframes of schools we have statistics on with the schools we do not have statistics on. This will be the complete dataset which has a list of all the high schools.**

In [23]:
# Combining the known school data with the unknown school data 
all_high_school_stats = pd.concat([known_high_school_stats, unknown_high_schools], sort=False)

# Resetting the Index
all_high_school_stats = all_high_school_stats.reset_index().drop(columns = ['index'])

# Test
all_high_school_stats_test = (len(all_high_school_stats) == (len(known_high_school_stats) + len(unknown_high_schools)))

if(all_high_school_stats_test):
    print("PASSED TEST: ALL HIGH SCHOOLS TEST")
else: 
    print("NOT PASSED TEST: ALL HIGH SCHOOLS TEST")

# Printing the Head
all_high_school_stats.head()

PASSED TEST: ALL HIGH SCHOOLS TEST


,SchoolCode,SchoolName,school_teaches_cs,total_cs_courses,yearly_enrolled_in_cs
0,1519,Edmonds eLearning Academy,Teaches Computer Science,1.0,1.0
1,1547,Middle College High School,Teaches Computer Science,4.0,58.0
2,1627,Yelm Extension School,Teaches Computer Science,2.0,3.0
3,1628,Dishman Hills High School,Teaches Computer Science,6.0,158.0
4,1640,Puyallup Online Academy/POA,Teaches Computer Science,2.0,6.0


## Part 14: Importing and saving GeoData and School Metadata File

**For us to map these schools, we need to add key geo-spatial data such as latitude and longitude. We have sourced this geodata for K-12 schools from [Washington State Data Gov Website](https://geo.wa.gov/datasets/OSPI::k-12-schools).** 

**While this website claims to have information on all Washington State K-12 schools, this is not true. It was missing 8 schools for the 2017 dataset. These schools were:**

    1. Sage Hills High School
    2. Marysville Mountain View High School
    3. Nooksack Reengagement
    4. Selah Academy BPL
    5. College Place Open Doors Program
    6. Tonasket Choice High School
    7. Tonasket Outreach School
    8. Moses Lake Big Picture

**I manually added the information about these schools to the dataset used below. You can see the new additions by comparing the `WA_K12_Schools_Geo_Data_With_Manual_Additions.csv` file and the `WA_K12_Schools_Geo_Data.csv` file. For these additions I referred to the [Washington School directory](https://eds.ospi.k12.wa.us/directoryeds.aspx) and if I did not find information there then I looked at the OSPI website and Google. For the co-ordinates, I sourced them from Google Maps.**

**Below is a description of each column.**

| **Column Name** | **Column Description** | 
| --- | --- | 
| X | Longitude of the School Location |
| Y  | Latitude of the School Location  | 
| FID | Unique ID of the School in this dataset |
| School Code | Washing State School Code of this School |
| Latitude | Latitude of School | 
| Longitude | Longitude of School |
| ESDCode | Education Service District Code of the School | 
| ESDName | Education Service District Code of the School | 
| LEACode | Local Education Agency Code for the School | 
| SchoolName | Name of the School | 
| LowestGrad | Lowest Grade of the School |
| HighestGra | Highest Grade of the School |
| AddressLin | Address Line 1 of the School | 
| AddressL_1 | Address Line 2 of the School (optional)| 
| City | City of the School |
| State | State of the School | 
| ZipCode | ZipCode of the School | 
| PricipalN | Principal Name of the School |
| Email | Email ID of the Principal | 
| Phone | Phone Number of the School | 
| OrgCategor | Type of School by Organization Type e.g Public School, Re-Engagement School	 | 
| AYPCode | Adequate Yearly Progress Code | 
| GradeCateg | Grade Category of the School |
| OrgCateg_1 | Organization Category of the School |

In [24]:
#Importing Data
wa_school_geo_data = pd.read_csv("data/labelled_data/School_Data/WA_K12_Schools_Geo_Data_With_Manual_Additions.csv")

#Printing the Head of the Dataframe
wa_school_geo_data.head()

,X,Y,FID,SchoolCode,Latitude,Longitude,ESDCode,ESDName,LEACode,LEAName,...,City,State,ZipCode,PrincipalN,Email,Phone,OrgCategor,AYPCode,GradeCateg,OrgCateg_1
0,-119.195783,46.224367,2001,4007,46.224373,-119.195797,11801,Educational Service District 123,3017,Kennewick School District,...,KENNEWICK,Washington,99336-1300,Dennis Boatman,dennis.boatman@ksd.org,509.222.6522,Detention Center,J,Other,Public
1,-122.354846,47.211844,2002,5549,47.211850,-122.354860,OSPI,Office of Superintendent of Public Instruction,27901,Chief Leschi Tribal Compact,...,Puyallup,Washington,98371,Bruce Leonardy,bruce.leonardy@leschischools.org,253.445.6000,"Not Affiliated With District, Tribal School",Q,K-12,Tribal
2,-122.460763,45.593231,2003,5534,45.593237,-122.460777,6801,Educational Service District 112,6117,Camas School District,...,Camas,Washington,98607,Aaron J Smith,aaronj.smith@camas.wednet.edu,360-833-5780,"Affiliated With District, Public School",P,Middle School,Public
3,-117.558706,47.808964,2004,5417,47.808970,-117.558720,32801,Educational Service District 101,32325,Nine Mile Falls School District,...,Nine Mile Falls,Washington,99026,Willard B Osborn,bosborn@9mile.org,509.340.4200,"Public School, Re-Engagement School",R,High School,Public
4,-122.917265,46.994554,2005,5305,46.994560,-122.917280,OSPI,Office of Superintendent of Public Instruction,34801,Capital Region ESD 113,...,Tumwater,Washington,98512,Gerald Grubbs,ggrubbs@esd113.org,360.927.6232,"Public School, Re-Engagement School",R,High School,Public


## Part 15: Merging High School Statistics with the GeoData and Metadata Dataframe

**In this step, we are merging the school statistics (`all_high_school_stats`) with the geometa dataframe(`wa_school_geo_data`) created above in Part 14.**

*Note: The length of the all_high_stats is originally 660 rows. However on the merge our dataframe is 663 rows. This is because there are 3 special program schools which share the same school code as the school of that learning district.*

**IMP: Check the counts on the merge - school code, latitude, longitude should be equal to each other. I do this by printing the description of the dataframe.**

In [27]:
# Merging statistics to the geo and meta data
all_high_school_stats_geo_meta_data = pd.merge(all_high_school_stats, wa_school_geo_data, how = 'left', on = 'SchoolCode')

# checking the counts on the merge - school code, latitude, longitude should be equal to each other
all_high_school_stats_geo_meta_data.describe()

,SchoolCode,total_cs_courses,yearly_enrolled_in_cs,X,Y,FID,Latitude,Longitude,LEACode
count,663.000000,372.000000,372.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000
mean,3652.253394,3.814516,101.696237,-121.335308,47.348962,1464.092006,47.348968,-121.335323,22588.079940
std,1363.577206,4.408458,161.320600,1.891257,0.793857,730.116048,0.793857,1.891257,10399.099384
min,1500.000000,0.000000,0.000000,-124.637535,45.584954,5.000000,45.584960,-124.637550,1109.000000
25%,2393.500000,0.000000,0.000000,-122.521441,46.973756,876.500000,46.973762,-122.521456,17001.000000
50%,3508.000000,2.000000,28.000000,-122.226676,47.460363,1527.000000,47.460369,-122.226691,23311.000000
75%,5182.000000,6.000000,152.250000,-120.258761,47.838239,2128.500000,47.838245,-120.258775,31311.000000
max,5961.000000,23.000000,1814.000000,-117.040365,48.993756,2451.000000,48.993762,-117.040378,39901.000000


**2. The values in the School Name, Principal Name, Email, City are not consistent in casing. Some are upper case while some are in Title Case. We want our data in each column to be in a consistent format. Therefore we have fixed the casing of these columns for consistency.**

In [28]:
all_high_school_stats_geo_meta_data["SchoolName_x"] = all_high_school_stats_geo_meta_data["SchoolName_x"].str.title()
all_high_school_stats_geo_meta_data["SchoolName_y"] = all_high_school_stats_geo_meta_data["SchoolName_y"].str.title()
all_high_school_stats_geo_meta_data["PrincipalN"] = all_high_school_stats_geo_meta_data["PrincipalN"].str.title()
all_high_school_stats_geo_meta_data["Email"] = all_high_school_stats_geo_meta_data["Email"].str.lower()
all_high_school_stats_geo_meta_data["City"] = all_high_school_stats_geo_meta_data["City"].str.title()
all_high_school_stats_geo_meta_data["State"] = all_high_school_stats_geo_meta_data["State"].str.title()

all_high_school_stats_geo_meta_data.head()

,SchoolCode,SchoolName_x,school_teaches_cs,total_cs_courses,yearly_enrolled_in_cs,X,Y,FID,Latitude,Longitude,...,City,State,ZipCode,PrincipalN,Email,Phone,OrgCategor,AYPCode,GradeCateg,OrgCateg_1
0,1519,Edmonds Elearning Academy,Teaches Computer Science,1.0,1.0,-122.334678,47.810655,809,47.810660,-122.334693,...,Lynnwood,Washington,98036-7400,Katie Bjornstad,bjornstadk@edmonds.wednet.edu,425.431.1528,"Alternative School, Public School",A,High School,Public
1,1547,Middle College High School,Teaches Computer Science,4.0,58.0,-122.325875,47.706525,1579,47.706531,-122.325890,...,Seattle,Washington,98125,Elizabeth Mcfarland,emmcfarland@seattleschools.org,206.252.9905,"Public School, Regular School",P,High School,Public
2,1627,Yelm Extension School,Teaches Computer Science,2.0,3.0,-122.605731,46.942461,1302,46.942467,-122.605746,...,Yelm,Washington,98597-0476,Ryan Akiyama,ryan_akiyama@ycs.wednet.edu,360.458.7777,"Alternative School, Public School",A,High School,Public
3,1628,Dishman Hills High School,Teaches Computer Science,6.0,158.0,-117.260226,47.655578,707,47.655585,-117.260239,...,Spokane,Washington,99206-0000,Lauren House,lauren.house@wvsd.org,509.927.1100,"Alternative School, Public School",A,High School,Public
4,1640,Puyallup Online Academy/Poa,Teaches Computer Science,2.0,6.0,-122.301955,47.160668,1552,47.160673,-122.301969,...,Puyallup,Washington,98373,Adriana Julian,juliaac@puyallup.k12.wa.us,253.841.8630,"Alternative School, Public School",A,K-12,Public


**3. As you can notice, the column names are not perfect. For example, a column is called PrincipalNa and HighestGra. These are incomplete column names. We are going to fix these so as to make it easier for any person to understand.**

*Note: We are renaming Zipcode as ZipCode Long as many Zipcodes are 9 digit zipcodes. The first 5 digits indiciate the post office or delivery area. The next 4 digits represent a specific delivery route within that delivery area.*

In [31]:
# renaming column names
all_high_school_stats_geo_meta_data = all_high_school_stats_geo_meta_data.rename(columns = {'SchoolCode': 'School_Code',
                                                                                            'SchoolName_x':'School_Name',
                                                                                           'school_teaches_cs': 'School_Teaches_Computer_Science',
                                                                                           'total_cs_courses': 'Total_Computer_Science_Courses_Taught',
                                                                                           'yearly_enrolled_in_cs': 'Yearly_Enrolled_in_Computer_Science',
                                                                                           'HighestGra': 'Highest_Grade',
                                                                                           'LowestGrad':'Lowest_Grade',
                                                                                           'PrincipalN': 'Principal_Name',
                                                                                           'OrgCategor': 'Org_Category',
                                                                                           'AddressLin': 'Address_Line_1',
                                                                                           'AddressL_1':'Address_Line_2',
                                                                                           'OrgCateg_1':'Org_Category_2',
                                                                                           'ZipCode': 'Zip_Code_Long'})

In [32]:
# Checking if column names have changed as desired
all_high_school_stats_geo_meta_data.columns

Index(['School_Code', 'School_Name', 'School_Teaches_Computer_Science',
       'Total_Computer_Science_Courses_Taught',
       'Yearly_Enrolled_in_Computer_Science', 'X', 'Y', 'FID', 'Latitude',
       'Longitude', 'ESDCode', 'ESDName', 'LEACode', 'LEAName', 'SchoolName_y',
       'Lowest_Grade', 'Highest_Grade', 'Address_Line_1', 'Address_Line_2',
       'City', 'State', 'Zip_Code_Long', 'Principal_Name', 'Email', 'Phone',
       'Org_Category', 'AYPCode', 'GradeCateg', 'Org_Category_2'],
      dtype='object')

**4. The original ZipCode column provided to us had many zipcodes (not all) in their long form. To ensure there is consistency therefore, we are adding a  5 digit simplified Zipcode column.**

In [33]:
all_high_school_stats_geo_meta_data["Zip_Code"] = all_high_school_stats_geo_meta_data["Zip_Code_Long"]
all_high_school_stats_geo_meta_data["Zip_Code"] = all_high_school_stats_geo_meta_data.Zip_Code.str.slice(stop=5)
all_high_school_stats_geo_meta_data.head()

,School_Code,School_Name,School_Teaches_Computer_Science,Total_Computer_Science_Courses_Taught,Yearly_Enrolled_in_Computer_Science,X,Y,FID,Latitude,Longitude,...,State,Zip_Code_Long,Principal_Name,Email,Phone,Org_Category,AYPCode,GradeCateg,Org_Category_2,Zip_Code
0,1519,Edmonds Elearning Academy,Teaches Computer Science,1.0,1.0,-122.334678,47.810655,809,47.810660,-122.334693,...,Washington,98036-7400,Katie Bjornstad,bjornstadk@edmonds.wednet.edu,425.431.1528,"Alternative School, Public School",A,High School,Public,98036
1,1547,Middle College High School,Teaches Computer Science,4.0,58.0,-122.325875,47.706525,1579,47.706531,-122.325890,...,Washington,98125,Elizabeth Mcfarland,emmcfarland@seattleschools.org,206.252.9905,"Public School, Regular School",P,High School,Public,98125
2,1627,Yelm Extension School,Teaches Computer Science,2.0,3.0,-122.605731,46.942461,1302,46.942467,-122.605746,...,Washington,98597-0476,Ryan Akiyama,ryan_akiyama@ycs.wednet.edu,360.458.7777,"Alternative School, Public School",A,High School,Public,98597
3,1628,Dishman Hills High School,Teaches Computer Science,6.0,158.0,-117.260226,47.655578,707,47.655585,-117.260239,...,Washington,99206-0000,Lauren House,lauren.house@wvsd.org,509.927.1100,"Alternative School, Public School",A,High School,Public,99206
4,1640,Puyallup Online Academy/Poa,Teaches Computer Science,2.0,6.0,-122.301955,47.160668,1552,47.160673,-122.301969,...,Washington,98373,Adriana Julian,juliaac@puyallup.k12.wa.us,253.841.8630,"Alternative School, Public School",A,K-12,Public,98373


## Part 16: Importing and Saving County Data

**While we have been able to gather and add the Geodata and Statistics data of each school, we are missing the County Data. To gather this information I went to the [ArcGIS Public Schools Locations web page](https://hub.arcgis.com/datasets/87376bdb0cb3490cbda39935626f6604_0). Their dataset had available a list of all the schools in the United States, the school's zipcodes and their counties.**

*Note: Since I wanted only the schools in Washington, I filtered  for them within excel and removed any duplicate zipcodes. After this I only kept the ZipCode, CountyID, and County columns and removed the rest. After that I saved the zipcodes file as a new file called `WA_School_Counties_List.csv`* 

Below is a description of each column name and we have also printed the head of the dataframe for easy viewing. 

| **Column Name** | **Column Description** | 
| --- | --- | 
| Zip_Code | Zip Code e.g. 98010|
| County_Name | Name of County e.g. 53033|
| CountyID | Unique ID of County in USA e.g. King County|

In [34]:
# Importing and Saving the School County List
school_county_list = pd.read_csv("data/labelled_data/School_Data/WA_School_Counties_List.csv")
# Setting the Datatype of the Zipcode column from int to str for easy merging
school_county_list["Zip_Code"] = school_county_list["Zip_Code"].astype(str)
# Printing the head of the dataset
school_county_list.head()

,Zip_Code,CountyID,County_Name
0,98010,53033,King County
1,98022,53033,King County
2,99138,53019,Ferry County
3,98626,53015,Cowlitz County
4,98632,53015,Cowlitz County


## Part 17: Merging Statistics with Geo Data with School County Dataset

**1. In this step we are going to merge the statistics dataset with the school county dataset. This is being done as we want to be able to see schools by their county.**

In [35]:
all_high_school_stats_geo_county_meta_data = pd.merge(all_high_school_stats_geo_meta_data,school_county_list, how="left", on = "Zip_Code")
all_high_school_stats_geo_county_meta_data.describe()

,School_Code,Total_Computer_Science_Courses_Taught,Yearly_Enrolled_in_Computer_Science,X,Y,FID,Latitude,Longitude,LEACode,CountyID
count,663.000000,372.000000,372.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000
mean,3652.253394,3.814516,101.696237,-121.335308,47.348962,1464.092006,47.348968,-121.335323,22588.079940,53043.971342
std,1363.577206,4.408458,161.320600,1.891257,0.793857,730.116048,0.793857,1.891257,10399.099384,20.774667
min,1500.000000,0.000000,0.000000,-124.637535,45.584954,5.000000,45.584960,-124.637550,1109.000000,53001.000000
25%,2393.500000,0.000000,0.000000,-122.521441,46.973756,876.500000,46.973762,-122.521456,17001.000000,53033.000000
50%,3508.000000,2.000000,28.000000,-122.226676,47.460363,1527.000000,47.460369,-122.226691,23311.000000,53047.000000
75%,5182.000000,6.000000,152.250000,-120.258761,47.838239,2128.500000,47.838245,-120.258775,31311.000000,53061.000000
max,5961.000000,23.000000,1814.000000,-117.040365,48.993756,2451.000000,48.993762,-117.040378,39901.000000,53077.000000


**2. Below will be a list of missing zipcodes if there are any. This is because the County Dataset will not always have all the zips needed. So you may need to go to the `data/labelled_data/School_Data/` path and open the `WA_School_Counties_List.csv` file and manually add them to this file and save it and then re-run all the cells starting from Part 16. Only continue once you see that there are 0 missing zips.** 

In [36]:
# Running a query to see which zips are missing
missing_zips = all_high_school_stats_geo_county_meta_data[~all_high_school_stats_geo_county_meta_data.Zip_Code.isin(school_county_list.Zip_Code)]
# Print how many zips are missing
print("There are", len(missing_zips), "missing zip codes")

if(len(missing_zips)> 0):
    print("They are")
    print(missing_zips)

There are 0 missing zip codes


## Part 18: Saving and Writing the prepared visualized set to the Visualization Data Folder
**Now that the dataset is ready for a visualization I am going to save it in the `data/visualization_data/` path as a CSV file.**

**1. We are first going to check if all our column names are in the desired format**

In [37]:
all_high_school_stats_geo_county_meta_data.columns

Index(['School_Code', 'School_Name', 'School_Teaches_Computer_Science',
       'Total_Computer_Science_Courses_Taught',
       'Yearly_Enrolled_in_Computer_Science', 'X', 'Y', 'FID', 'Latitude',
       'Longitude', 'ESDCode', 'ESDName', 'LEACode', 'LEAName', 'SchoolName_y',
       'Lowest_Grade', 'Highest_Grade', 'Address_Line_1', 'Address_Line_2',
       'City', 'State', 'Zip_Code_Long', 'Principal_Name', 'Email', 'Phone',
       'Org_Category', 'AYPCode', 'GradeCateg', 'Org_Category_2', 'Zip_Code',
       'CountyID', 'County_Name'],
      dtype='object')

**2. Next we are going to see a sample of 10 rows from our dataset**

In [38]:
all_high_school_stats_geo_county_meta_data.sort_values('School_Code')
all_high_school_stats_geo_county_meta_data.sample(10)

,School_Code,School_Name,School_Teaches_Computer_Science,Total_Computer_Science_Courses_Taught,Yearly_Enrolled_in_Computer_Science,X,Y,FID,Latitude,Longitude,...,Principal_Name,Email,Phone,Org_Category,AYPCode,GradeCateg,Org_Category_2,Zip_Code,CountyID,County_Name
642,5181,Snoqualmie Access,No Information Available,NaN,NaN,-121.827913,47.528216,1596,47.528222,-121.827927,...,Nancy Meeks,meeksn@svsd410.org,425.831.8015,"Public School, Special Education School",S,PK-12,Public,98065,53033,King County
393,1983,Lynden Academy,No Information Available,NaN,NaN,-122.462714,48.938413,1465,48.938419,-122.462729,...,Ellie Meenk,meenke@lynden.wednet.edu,360.354.6675,"Alternative School, Public School",A,K-12,Public,98264,53073,Whatcom County
632,5107,Federal Way Running Start Home School,No Information Available,NaN,NaN,-122.324636,47.304898,1402,47.304904,-122.324651,...,Sammy Anderson,samander@fwps.org,253.945.2000,"Public School, Regular School",P,High School,Public,98003,53033,King County
274,4020,Yakima Valley Technical Skills Center,Doesn't Teach Computer Science,0.0,0.0,-120.478463,46.589522,2321,46.589528,-120.478477,...,Dennis Matson,matson.dennis@yakimaschools.org,509.573.5501,"Public School, Vocational/technical school",V,High School,Public,98901,53077,Yakima County
411,3932,Lewis And Clark High School,No Information Available,NaN,NaN,-122.620616,45.642581,2088,45.642587,-122.620630,...,Steven D Lindblom,steve.lindblom@vansd.org,360.313.4350,"Public School, Regular School",P,High School,Public,98661,53011,Clark County
182,4247,Aces High School,Teaches Computer Science,4.0,42.0,-122.244363,47.910693,417,47.910699,-122.244378,...,Amy D Montanye-Johnson,montanyejohnsonad@mukilteo.wednet.edu,425.366.3900,"Alternative School, Public School",A,High School,Public,98204,53061,Snohomish County
403,1612,Skill Source,No Information Available,NaN,NaN,-120.316452,47.427326,2108,47.427332,-120.316466,...,Kory Kalahar,kalahar.k@wenatcheeschools.org,509.663.3369,"Alternative School, Public School",A,High School,Public,98801,53007,Chelan County
422,5317,Lewis County Jail,No Information Available,NaN,NaN,-122.969163,46.660612,2431,46.660618,-122.969177,...,Tim Touhey,ttouhey@chehalisschools.org,360.740.3521,Institution,I,High School,Public,98532,53041,Lewis County
388,1962,Garfield At Palouse High School,No Information Available,NaN,NaN,-117.071057,46.912635,1410,46.912641,-117.071071,...,Zane Wells,zwells@garpal.wednet.edu,509.878.1921,"Public School, Regular School",P,High School,Public,99161,53075,Whitman County
204,5033,Graham Kapowsin High School,Teaches Computer Science,12.0,342.0,-122.286737,47.057330,972,47.057336,-122.286751,...,Matt Yarkosky,myarkosky@bethelsd.org,253.683.6100,"Public School, Regular School",P,High School,Public,98338,53053,Pierce County


**3. We are now going to see the counts for our key columns: SchoolCode, County, Latitude and Longitude. They should all be equal to each other as well as to the total rows of the dataset.**

In [39]:
print("Total Rows are: ", len(all_high_school_stats_geo_county_meta_data))

Total Rows are:  663


In [40]:
all_high_school_stats_geo_county_meta_data.describe()

,School_Code,Total_Computer_Science_Courses_Taught,Yearly_Enrolled_in_Computer_Science,X,Y,FID,Latitude,Longitude,LEACode,CountyID
count,663.000000,372.000000,372.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000
mean,3652.253394,3.814516,101.696237,-121.335308,47.348962,1464.092006,47.348968,-121.335323,22588.079940,53043.971342
std,1363.577206,4.408458,161.320600,1.891257,0.793857,730.116048,0.793857,1.891257,10399.099384,20.774667
min,1500.000000,0.000000,0.000000,-124.637535,45.584954,5.000000,45.584960,-124.637550,1109.000000,53001.000000
25%,2393.500000,0.000000,0.000000,-122.521441,46.973756,876.500000,46.973762,-122.521456,17001.000000,53033.000000
50%,3508.000000,2.000000,28.000000,-122.226676,47.460363,1527.000000,47.460369,-122.226691,23311.000000,53047.000000
75%,5182.000000,6.000000,152.250000,-120.258761,47.838239,2128.500000,47.838245,-120.258775,31311.000000,53061.000000
max,5961.000000,23.000000,1814.000000,-117.040365,48.993756,2451.000000,48.993762,-117.040378,39901.000000,53077.000000


**4. Finally saving and writing the dataset to the visualization data folder**

In [42]:
#Writing the dataset to the Visualization Data Folder
all_high_school_stats_geo_county_meta_data.to_csv("data/visualization_data/wa_school_statistics_power_bi_viz.csv", index = False, header=True)